In [ ]:
#!pip install -U pyvisa
#!pip install pymeasure
#!pip install pyvisa-py
#!pip install pyusb

In [ ]:
import pyvisa as visa
import time
import numpy as np
import pymeasure
from pymeasure.instruments.keithley import Keithley2400
import pandas as pd
import matplotlib.pyplot as plt
import datetime

# %matplotlib inline
%matplotlib widget

rm = visa.ResourceManager()
rm.list_resources()

In [ ]:
# type down the FOLDER you want the file to save to
file_dir = ""

In [ ]:
# Lock in
sensitivity = {
    1: 26,
    0.5: 25,
    0.2: 24,
    0.1: 23,
    0.05: 22,
    0.02: 21,
    0.01: 20,
    0.005: 19,
    0.002: 18,
    0.001: 17,
    0.0005: 16,
    0.0002: 15,
    0.0001: 14,
    0.00005: 13,
    0.00002: 12,
    0.00001: 11,
    0.000005: 10,
    0.000002: 9,
    0.000001: 8,
    0.0000005: 7,
    0.0000002: 6,
    0.0000001: 5,
    0.00000005: 4,
    0.00000002: 3,
    0.00000001: 2,
    0.000000005: 1,
    0.000000002: 0,
}


class Lockin:

    def __init__(self, resource, gpib=1):
        self.lock_in = resource.open_resource("GPIB0::{}::INSTR".format(gpib))

    #      self.lock_in.query('OUTX1;*IDN?')
    #      self.lock_in.write('*RST')
    #      self.lock_in.write('ALRM 0;KCLK 0;*ESE 53;ERRE 246;LIAE 7;*CLS;')

    def change_harmonic(self, harm=1, sens=1):
        self.lock_in.write("HARM {}".format(harm))
        self.lock_in.write("SENS {}".format(sensitivity[sens]))

    def lock_in_read(self):
        x = float(self.lock_in.query("OUTP? 1").strip())
        y = float(self.lock_in.query("OUTP? 2").strip())
        r = float(self.lock_in.query("OUTP? 3").strip())
        theta = float(self.lock_in.query("OUTP? 4").strip())
        return x, y, r, theta

    def lock_in_measure(self,
                        count=10,
                        time_step=0.1,
                        wait_before_measure=1,
                        harm=1,
                        sens=0.00001):
        #        self.change_harmonic(harm, sens)
        time.sleep(wait_before_measure)
        xs, ys, rs, thetas = [], [], [], []
        for i in range(count):
            x, y, r, theta = self.lock_in_read()
            xs += [x]
            ys += [y]
            rs += [r]
            thetas += [theta]
            time.sleep(time_step)

        xs = sorted(xs)
        ys = sorted(ys)
        rs = sorted(rs)
        thetas = sorted(thetas)

        if len(xs) > 30:
            xs = xs[round(1 / 6 * len(xs)):-round(1 / 6 * len(xs))]
            ys = ys[round(1 / 6 * len(ys)):-round(1 / 6 * len(ys))]
            rs = rs[round(1 / 6 * len(rs)):-round(1 / 6 * len(rs))]
            thetas = thetas[round(1 / 6 *
                                  len(thetas)):-round(1 / 6 * len(thetas))]

        x_mean, x_std = np.mean(xs), np.std(xs)
        y_mean, y_std = np.mean(ys), np.std(ys)
        r_mean, r_std = np.mean(rs), np.std(rs)
        theta_mean, theta_std = np.mean(thetas), np.std(thetas)

        return x_mean, y_mean, r_mean, theta_mean, x_std, y_std, r_std, theta_std


In [ ]:
# Initialize
Kepco = rm.open_resource("GPIB0::2::INSTR")
Delayline = rm.open_resource("ASRL6::INSTR", baud_rate=9600)
lock1 = Lockin(rm, 1)
lock2 = Lockin(rm, 8)
Fluke = rm.open_resource("GPIB0::6::INSTR")
# Oscilloscope=rm.open_resource( 'USB0::0x0957::0x172D::MY50340112::INSTR')

# FLUKE
Fluke.write("*")  # reset

# Oscilloscope
# def toBinary(a):
# l,m=[],[]
# for i in a:
# l.append(ord(i))
# for i in l:
# m.append(int(bin(i)[2:], 2))

# return m

# Kepco

Kepco.write("FUNC:MODE CURR")
# Kepco.write("OUTP ON;CURR 0")


In [ ]:
def Setfield(magcurr=0):
    Kepco.write("CURR {}".format(magcurr))
    Flag = False
    while not Flag:
        r = Kepco.query("MEAS:CURR?")
        readcurrent = float(r.split('\n')[0])
        if np.abs(magcurr - readcurrent) <= 0.1:
            Flag = True
    s = Fluke.query('Y1T0')
    Field = float(s.split(',')[0])*10000  # Oe
    print('kepco current is {} A;'.format(
        magcurr), 'field is {} Oe'.format(Field))
    return Field


def SetDelayline(t=100):
    writet = t-200           # unit ps
    writep = round(writet*1e-12*299792458/1e-6/2*10)/10
    writepstr = 'PX'+str(writep)
    Delayline.write(writepstr)
    time.sleep(1)

    not_succeeded, run_count = True, 0
    while not_succeeded:
        try:
            Delayline.write('DX')
            s = Delayline.read('DX', encoding='unicode_escape')
            float((s.split('=')[1]).split(' ')[0])
            readp = float((s.split('=')[1]).split(' ')[0])
            readt = readp*1e-6*2/299792458/1e-12                    # unit ps
            time.sleep(1)
            if np.abs(writep - readp) <= 0.1:
                not_succeeded = False
        except Exception as e:
            run_count += 1
            print(e)
            print("Run Count", run_count)
            time.sleep(0.3)

    print('delayline position is {};'.format(
        readp), 'time is {} ps'.format(readt+200))

    return readp, readt+200


def OscilloscpeProcess(Oscilloscope=Oscilloscpe):
    # Oscilloscope.write('*RST') #reset
    # Oscilloscope.write(':AUT') #auto scale
    # Oscilloscope.write(':DIG') #Initiate single snap
    Osi = Oscilloscope.query(':WAV:SOUR CHAN1;PRE?')  # channel 1
    dt = float(Osi.split(',')[4])
    dy = float(Osi.split(',')[7])
    TT = np.arange(0, 1000, 1)*dt
    Oscilloscope.write(':WAV:DATA?')  # channel 1
    str1 = Oscilloscope.read(':WAV:DATA?', encoding='unicode_escape')
    Binstr = toBinary(str1)[10:2010]
    Y = []
    for i in range(0, 1000):
        Y.append((Binstr[2*i]*256+Binstr[2*i+1]-32768)*dy)

    data = sum(Y)
    return data


def OscilloscpeProcess2(avg=100):
    Y = []
    for i in range(avg):
        Y.append(OscilloscpeProcess())
    Y = np.sort(Y)
#    plt.plot(Y)
    data = np.mean(Y[round(3*len(Y)/4)-5:round(3*len(Y)/4)+5]) - \
        np.mean(Y[round(len(Y)/4)-5:round(len(Y)/4)+5])

    return data


def FielddepMeaasurment(ID='test', currs=[], count=1, Field=True, Reset=False):

    file_name = "C:/User Data/Hanshen Huang/LFO(110)_221215/"+ID+"_Kepco_{}A_to_{}A_{}.csv".format(
        min(currs), max(currs), int(time.time()))
    with open(file_name, 'w') as f:
        f.write("Current, lockin1x, lockin1y, lockin2x, lockin2y, Field'\n")

    # Interactive Mode
    %matplotlib notebook
    # Create subplots
    figure, axes = plt.subplots(4, 1, figsize=(5, 20))
    plt.subplots_adjust(left=0.2, bottom=0.1, right=0.9,
                        top=0.9, wspace=0.4, hspace=0.4)
    # Data Coordinates
    x1, x2, x3, x4, y1, y2, y3, y4, field = [], [], [], [], [], [], [], [], []
    # GUI
    plt.ion()
    #  Plot
    plot1, = axes[0].plot(x1, y1, 'bo-')
    plot2, = axes[1].plot(x2, y2, 'go-')
    plot3, = axes[2].plot(x3, y3, 'bo-')
    plot4, = axes[3].plot(x4, y4, 'go-')
    # Labels
    if Field:
        axes[0].set_xlabel("Field", fontsize=18)
        axes[1].set_xlabel("Field", fontsize=18)
        axes[2].set_xlabel("Field", fontsize=18)
        axes[3].set_xlabel("Field", fontsize=18)
    else:
        axes[0].set_xlabel("Kepco Current", fontsize=18)
        axes[1].set_xlabel("Kepco Current", fontsize=18)
        axes[2].set_xlabel("Kepco Current", fontsize=18)
        axes[3].set_xlabel("Kepco Current", fontsize=18)

    axes[0].set_ylabel("Lockin1_X", fontsize=18)
    axes[1].set_ylabel("Lockin1_Y", fontsize=18)
    axes[2].set_ylabel("Lockin2_X", fontsize=18)
    axes[3].set_ylabel("Lockin2_Y", fontsize=18)

    for curr in currs:
        F = Setfield(magcurr=curr)
        A = lock1.lock_in_measure(
            count=count, time_step=0.1, wait_before_measure=0.0)
        B = lock2.lock_in_measure(
            count=count, time_step=0.1, wait_before_measure=0.0)

        if Field:
            x1.append(F)
            x2.append(F)
            x3.append(F)
            x4.append(F)
        else:
            x1.append(curr)
            x2.append(curr)
            x3.append(curr)
            x4.append(curr)

        y1.append(A[0])
        y2.append(A[1])
        y3.append(B[0])
        y4.append(B[1])
        field.append(F)

        plot1.set_xdata(x1)
        plot1.set_ydata(y1)
        axes[0].relim()
        axes[0].autoscale_view(True, True, True)
        figure.canvas.draw()
        figure.canvas.flush_events()
        plt.show()

        plot2.set_xdata(x2)
        plot2.set_ydata(y2)
        axes[1].relim()
        axes[1].autoscale_view(True, True, True)
        figure.canvas.draw()
        figure.canvas.flush_events()
        plt.show()

        plot3.set_xdata(x3)
        plot3.set_ydata(y3)
        axes[2].relim()
        axes[2].autoscale_view(True, True, True)
        figure.canvas.draw()
        figure.canvas.flush_events()
        plt.show()

        plot4.set_xdata(x4)
        plot4.set_ydata(y4)
        axes[3].relim()
        axes[3].autoscale_view(True, True, True)
        figure.canvas.draw()
        figure.canvas.flush_events()
        plt.show()

        with open(file_name, 'a') as f:
            f.write("{},{},{},{},{},{}\n".format(
                curr, A[0], A[1], B[0], B[1], F))

        plt.savefig(file_name+'.png')
    if Reset:
        Setfield(magcurr=0)


def TimedepMeaasurment(ID='test', ts=[], curr=0, count=1, Reset=False, Pos=False):

    F = Setfield(magcurr=curr)

    file_name = file_dir+ID+"_Delayline_{}ps_to_{}ps_{}Oe_{}.csv".format(
        min(ts), max(ts), F, int(time.time()))
    with open(file_name, 'w') as f:
        f.write("Time, lockin1x, lockin1y, lockin2x, lockin2y, Field, Position'\n")

    # Create subplots
    figure, axes = plt.subplots(4, 1, figsize=(5, 20))
    plt.subplots_adjust(left=0.2, bottom=0.1, right=0.9,
                        top=0.9, wspace=0.4, hspace=0.4)
    # Data Coordinates
    x1, x2, x3, x4, y1, y2, y3, y4, delay, position = [], [], [], [], [], [], [], [], [], []
    # GUI
    plt.ion()
    #  Plot
    plot1, = axes[0].plot(x1, y1, 'bo-')
    plot2, = axes[1].plot(x2, y2, 'go-')
    plot3, = axes[2].plot(x3, y3, 'bo-')
    plot4, = axes[3].plot(x4, y4, 'go-')
    # Labels
    if Pos:
        axes[0].set_xlabel("Position", fontsize=18)
        axes[1].set_xlabel("Position", fontsize=18)
        axes[2].set_xlabel("Position", fontsize=18)
        axes[3].set_xlabel("Position", fontsize=18)
    else:
        axes[0].set_xlabel("Time", fontsize=18)
        axes[1].set_xlabel("Time", fontsize=18)
        axes[2].set_xlabel("Time", fontsize=18)
        axes[3].set_xlabel("Time", fontsize=18)
    axes[0].set_ylabel("Lockin1_X", fontsize=18)
    axes[1].set_ylabel("Lockin1_Y", fontsize=18)
    axes[2].set_ylabel("Lockin2_X", fontsize=18)
    axes[3].set_ylabel("Lockin2_Y", fontsize=18)

    for t in ts:
        T = SetDelayline(t=t)
        A = lock1.lock_in_measure(
            count=count, time_step=0.1, wait_before_measure=0.0)
        B = lock2.lock_in_measure(
            count=count, time_step=0.1, wait_before_measure=0.0)

        if Pos:
            x1.append(T[0])
            x2.append(T[0])
            x3.append(T[0])
            x4.append(T[0])
        else:
            x1.append(T[1])
            x2.append(T[1])
            x3.append(T[1])
            x4.append(T[1])

        y1.append(A[0])
        y2.append(A[1])
        y3.append(B[0])
        y4.append(B[1])
        delay.append(T[1])
        position.append(T[0])

        plot1.set_xdata(x1)
        plot1.set_ydata(y1)
        axes[0].relim()
        axes[0].autoscale_view(True, True, True)
        figure.canvas.draw()
        figure.canvas.flush_events()
        plt.show()

        plot2.set_xdata(x2)
        plot2.set_ydata(y2)
        axes[1].relim()
        axes[1].autoscale_view(True, True, True)
        figure.canvas.draw()
        figure.canvas.flush_events()
        plt.show()

        plot3.set_xdata(x3)
        plot3.set_ydata(y3)
        axes[2].relim()
        axes[2].autoscale_view(True, True, True)
        figure.canvas.draw()
        figure.canvas.flush_events()
        plt.show()

        plot4.set_xdata(x4)
        plot4.set_ydata(y4)
        axes[3].relim()
        axes[3].autoscale_view(True, True, True)
        figure.canvas.draw()
        figure.canvas.flush_events()
        plt.show()

        with open(file_name, 'a') as f:
            f.write("{},{},{},{},{},{},{}\n".format(
                t, A[0], A[1], B[0], B[1], F, T[0]))
        plt.savefig(file_name+'.png')
        plt.show()

    if Reset:
        Setfield(magcurr=0)


In [ ]:
def FielddepMeaasurmentwithOscilloscope(ID='test', currs=[], curr=0, count=1, Field=True, Reset=False):

    file_name = file_dir+ID + \
        "_Kepco_{}A_to_{}A_{}.csv".format(
            min(currs), max(currs), int(time.time()))
    with open(file_name, 'w') as f:
        f.write(
            "Current, lockin1x, lockin1y, lockin2x, lockin2y, Oscilloscope, Field\n")

    # Create subplots
    figure, axes = plt.subplots(4, 1, figsize=(5, 20))
    plt.subplots_adjust(left=0.2, bottom=0.1, right=0.8,
                        top=0.9, wspace=0.4, hspace=0.4)
    # Data Coordinates
    x1, y1, y2, y3, y4, y5, field = [], [], [], [], [], [], []
    # GUI
    plt.ion()
    #  Plot
    plot1, = axes[0].plot(x1, y1, 'ro-')
    plot2, = axes[1].plot(x1, y1, 'ro-')
    plot3, = axes[2].plot(x1, y1, 'ro-')
    plot4, = axes[3].plot(x1, y1, 'ro-')
    # Labels
    if Field:
        xlabel = "Field"
    else:
        xlabel = "Kepco Current"

    axes[0].set_xlabel(xlabel, fontsize=18)
    axes[0].set_ylabel("Oscilloscope", color="red", fontsize=18)
    axes2 = axes[0].twinx()
    axes2.set_ylabel("Lockin1x", color="blue", fontsize=18)

    axes[1].set_xlabel(xlabel, fontsize=18)
    axes[1].set_ylabel("Oscilloscope", color="red", fontsize=18)
    axes3 = axes[1].twinx()
    axes3.set_ylabel("Lockin1y", color="blue", fontsize=18)

    axes[2].set_xlabel(xlabel, fontsize=18)
    axes[2].set_ylabel("Oscilloscope2", color="red", fontsize=18)
    axes4 = axes[2].twinx()
    axes4.set_ylabel("Lockin2x", color="blue", fontsize=18)

    axes[3].set_xlabel(xlabel, fontsize=18)
    axes[3].set_ylabel("Oscilloscope2", color="red", fontsize=18)
    axes5 = axes[3].twinx()
    axes5.set_ylabel("Lockin2y", color="blue", fontsize=18)

    for curr in currs:
        F = Setfield(magcurr=curr)
        if Field:
            x1.append(F)
        else:
            x1.append(curr)

        data = OscilloscpeProcess()
        A = lock1.lock_in_measure(
            count=count, time_step=0.1, wait_before_measure=0.0)
        B = lock2.lock_in_measure(
            count=count, time_step=0.1, wait_before_measure=0.0)
        y1.append(data)
        y2.append(A[0])
        y3.append(A[1])
        y4.append(B[0])
        y5.append(B[1])
        field.append(F)

        axes[0].set_xlim([min(x1), max(x1)])
        axes[0].set_ylim([min(y1), max(y1)])
        axes2.set_ylim([min(y2), max(y2)])
        axes[0].plot(x1, y1, 'ro-')
        axes2.plot(x1, y2, 'bo-')
        figure.canvas.draw()
        figure.canvas.flush_events()
        plt.show()

        axes[1].set_xlim([min(x1), max(x1)])
        axes[1].set_ylim([min(y1), max(y1)])
        axes3.set_ylim([min(y3), max(y3)])
        axes[1].plot(x1, y1, 'ro-')
        axes3.plot(x1, y3, 'bo-')
        figure.canvas.draw()
        figure.canvas.flush_events()
        plt.show()

        axes[2].set_xlim([min(x1), max(x1)])
        axes[2].set_ylim([min(y1), max(y1)])
        axes4.set_ylim([min(y4), max(y4)])
        axes[2].plot(x1, y1, 'ro-')
        axes4.plot(x1, y4, 'bo-')
        figure.canvas.draw()
        figure.canvas.flush_events()
        plt.show()

        axes[3].set_xlim([min(x1), max(x1)])
        axes[3].set_ylim([min(y1), max(y1)])
        axes5.set_ylim([min(y5), max(y5)])
        axes[3].plot(x1, y1, 'ro-')
        axes5.plot(x1, y5, 'bo-')
        figure.canvas.draw()
        figure.canvas.flush_events()
        plt.show()
        with open(file_name, 'a') as f:
            f.write("{},{},{},{},{},{},{}\n".format(
                curr, A[0], A[1], B[0], B[1], data, F))
        plt.savefig(file_name+'.png')

    if Reset:
        Setfield(magcurr=0)


def TimedepMeaasurmentwithOscilloscope(ID='test', ts=[], curr=0, count=1, avg=100, Reset=False, Pos=False):

    F = Setfield(magcurr=curr)

    file_name = file_dir+ID+"_Delayline_{}ps_to_{}ps_{}Oe_{}.csv".format(
        min(ts), max(ts), F, int(time.time()))
    with open(file_name, 'w') as f:
        f.write(
            "Time, lockin1x, lockin1y, lockin2x, lockin2y, Oscilloscope, dOscilloscope, Field, Position\n")

    # Create subplots
    figure, axes = plt.subplots(4, 1, figsize=(5, 20))
    plt.subplots_adjust(left=0.2, bottom=0.1, right=0.8,
                        top=0.9, wspace=0.4, hspace=0.4)
    # Data Coordinates
    x1, y1, y12, y2, y3, y4, y5, delay, position = [], [], [], [], [], [], [], [], []
    # GUI
    plt.ion()
    #  Plot
    plot1, = axes[0].plot(x1, y1, 'ro-')
    plot2, = axes[1].plot(x1, y1, 'ro-')
    plot3, = axes[2].plot(x1, y12, 'ro-')
    plot4, = axes[3].plot(x1, y12, 'ro-')
    # Labels
    if Pos:
        xlabel = "Position"
    else:
        xlabel = "Time"

    axes[0].set_xlabel(xlabel, fontsize=18)
    axes[0].set_ylabel("Oscilloscope", color="red", fontsize=18)
    axes2 = axes[0].twinx()
    axes2.set_ylabel("Lockin1x", color="blue", fontsize=18)

    axes[1].set_xlabel(xlabel, fontsize=18)
    axes[1].set_ylabel("Oscilloscope", color="red", fontsize=18)
    axes3 = axes[1].twinx()
    axes3.set_ylabel("Lockin1y", color="blue", fontsize=18)

    axes[2].set_xlabel(xlabel, fontsize=18)
    axes[2].set_ylabel("Oscilloscope2", color="red", fontsize=18)
    axes4 = axes[2].twinx()
    axes4.set_ylabel("Lockin2x", color="blue", fontsize=18)

    axes[3].set_xlabel(xlabel, fontsize=18)
    axes[3].set_ylabel("Oscilloscope2", color="red", fontsize=18)
    axes5 = axes[3].twinx()
    axes5.set_ylabel("Lockin2y", color="blue", fontsize=18)

    for t in ts:
        T = SetDelayline(t=t)
        if Pos:
            x1.append(T[0])
        else:
            x1.append(T[1])
        data = OscilloscpeProcess()
        data2 = OscilloscpeProcess2(avg=avg)
        A = lock1.lock_in_measure(
            count=count, time_step=0.1, wait_before_measure=0.0)
        B = lock2.lock_in_measure(
            count=count, time_step=0.1, wait_before_measure=0.0)
        y1.append(data)
        y12.append(data2)
        y2.append(A[0])
        y3.append(A[1])
        y4.append(B[0])
        y5.append(B[1])
        delay.append(T[1])
        position.append(T[0])

        axes[0].set_xlim([min(x1), max(x1)])
        axes[0].set_ylim([min(y1), max(y1)])
        axes2.set_ylim([min(y2), max(y2)])
        axes[0].plot(x1, y1, 'ro-')
        axes2.plot(x1, y2, 'bo-')
        figure.canvas.draw()
        figure.canvas.flush_events()
        plt.show()

        axes[1].set_xlim([min(x1), max(x1)])
        axes[1].set_ylim([min(y1), max(y1)])
        axes3.set_ylim([min(y3), max(y3)])
        axes[1].plot(x1, y1, 'ro-')
        axes3.plot(x1, y3, 'bo-')
        figure.canvas.draw()
        figure.canvas.flush_events()
        plt.show()

        axes[2].set_xlim([min(x1), max(x1)])
        axes[2].set_ylim([min(y12), max(y12)])
        axes4.set_ylim([min(y4), max(y4)])
        axes[2].plot(x1, y12, 'ro-')
        axes4.plot(x1, y4, 'bo-')
        figure.canvas.draw()
        figure.canvas.flush_events()
        plt.show()

        axes[3].set_xlim([min(x1), max(x1)])
        axes[3].set_ylim([min(y12), max(y12)])
        axes5.set_ylim([min(y5), max(y5)])
        axes[3].plot(x1, y12, 'ro-')
        axes5.plot(x1, y5, 'bo-')
        figure.canvas.draw()
        figure.canvas.flush_events()
        plt.show()
        with open(file_name, 'a') as f:
            f.write("{},{},{},{},{},{},{},{},{}\n".format(
                t, A[0], A[1], B[0], B[1], data, data2, F, T[0]))
        plt.savefig(file_name+'.png')

    if Reset:
        Setfield(magcurr=0)


In [ ]:
#test lockin
lock1 = Lockin(rm, 1)
lock2 = Lockin(rm, 8)
A = lock1.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0.0)
B = lock2.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0.0)
print(A)
print(B)

In [ ]:
#set field
curr = 0
Setfield(magcurr=curr)

In [ ]:
#set delayline
t = 100
SetDelayline(t=t)

In [ ]:
# Field dep no Oscilloscope

mincurr = 35
maxcurr = -35
step = -1
count = 3
currs = np.append(
    np.arange(mincurr, maxcurr + step, step), np.arange(maxcurr, mincurr - step, -step)
)
print(currs)
FielddepMeaasurment(ID="10uW_400nm", currs=currs, count=count, Field=False, Reset=False)


In [ ]:
# Time dep no Oscilloscope

start = 100
stop = 500
step = 1
count = 3
ts = np.arange(start, stop + step, step)
print(ts)
if min(ts) > 50 and min(ts) < 800:
    TimedepMeaasurment(
        ID="M01_kvsdt_probe=800nm13uW90LP_pump=400nm250uW90LP_45Deg_0mA",
        ts=ts,
        curr=curr,
        count=count,
        Reset=False,
        Pos=False,
    )
else:
    print("ts wrong")


In [ ]:
# Field dep with Oscilloscope

mincurr = -35
maxcurr = 35
step = 2
count = 10
currs = np.append(
    np.arange(mincurr, maxcurr + step, step), np.arange(maxcurr, mincurr - step, -step)
)
print(currs)
FielddepMeaasurmentwithOscilloscope(
    ID="test", currs=currs, count=count, Field=False, Reset=False
)


In [ ]:
# Time dep with Oscilloscope

start = 80
stop = 90
step = 2
curr = 30
count = 10
avg = 100
ts = np.arange(start, stop + step, step)
print(ts)
if min(ts) > 50 and min(ts) < 800:
    TimedepMeaasurmentwithOscilloscope(
        ID="test", ts=ts, curr=curr, count=count, avg=avg, Reset=False, Pos=False
    )
else:
    print("ts wrong")
